In [36]:
import pandas as pd
from math import log, sqrt
from sklearn import linear_model  # using scikit-learn
import numpy as np

dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)
testing = pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('wk3_kc_house_valid_data.csv', dtype=dtype_dict)


In [29]:
def add_features(df):
    df['sqft_living_sqrt'] = df['sqft_living'].apply(sqrt)
    df['sqft_lot_sqrt'] = df['sqft_lot'].apply(sqrt)
    df['bedrooms_square'] = df['bedrooms']*df['bedrooms']
    df['floors_square'] = df['floors']*df['floors']

In [30]:
# add features to all data sets
for df in [sales, testing, training, validation]:
    add_features(df)
    
all_features = [
    'bedrooms', 'bedrooms_square',
    'bathrooms',
    'sqft_living', 'sqft_living_sqrt',
    'sqft_lot', 'sqft_lot_sqrt',
    'floors', 'floors_square',
    'waterfront', 'view', 'condition', 'grade',
    'sqft_above',
    'sqft_basement',
    'yr_built', 'yr_renovated']

In [48]:
# Lasso with L1 = 5e2
model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights
# Features chosen by LASSO
[all_features[c] for c in [i for i, x in enumerate(model_all.coef_ !=0) if x]]

['sqft_living', 'view', 'grade']

In [49]:
# Find a good L1 penalty value using a validation set
l1_penalties = np.logspace(1, 7, num=13)
results = []
for l1 in l1_penalties:
    # Learn a model on TRAINING data using the specified l1_penalty
    model = linear_model.Lasso(alpha=l1, normalize=True)
    model.fit(training[all_features], training['price'])
    # Compute the RSS on VALIDATION for the current model
    rss = sum((validation['price'] - model.predict(validation[all_features]))**2)
    results.append({'l1': l1, 'rss': rss})

pd.DataFrame(results).sort_values('rss').iloc[0]['l1']

10.0

In [72]:
# Compute the RSS on TEST data for the model with the best L1 penalty selected above with lowest RSS
final_model = linear_model.Lasso(alpha=10, normalize=True)
final_model.fit(training[all_features], training['price'])
rss = sum((testing['price'] - final_model.predict(testing[all_features]))**2)
print('Test RSS: ', rss)

# Using the best L1 penalty, how many nonzero weights do you have? 
# Count the number of nonzero coefficients first, and add 1 if the intercept is also nonzero
np.count_nonzero(final_model.coef_) + np.count_nonzero(final_model.intercept_)

Test RSS:  98467402552698.92


15

#### Limiting the model to a set number of features #### 
- Explore a large range of ‘l1_penalty’ values to find a narrow region of ‘l1_penalty’ values where models are likely to have the desired number of non-zero weights.
- Further explore the narrow region you found to find a good value for ‘l1_penalty’ that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for ‘l1_penalty’.

Out of this large range, we want to find the two ends of our desired narrow range of l1_penalty. At one end, we will have l1_penalty values that have too few non-zeros, and at the other end, we will have an l1_penalty that has too many non-zeros.

More formally, find:
- The largest l1_penalty that has more non-zeros than ‘max_nonzeros’ (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)Store this value in the variable ‘l1_penalty_min’ (we will use it later)
- The smallest l1_penalty that has fewer non-zeros than ‘max_nonzeros’ (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)Store this value in the variable ‘l1_penalty_max’ (we will use it later)

Hint: there are many ways to do this, e.g.:
- Programmatically within the loop above
- Creating a list with the number of non-zeros for each value of l1_penalty and inspecting it to find the appropriate boundaries.

In [51]:
max_nonzeros = 7

more_nonzeros = []
fewer_nonzeros = []

for l1 in np.logspace(1, 4, num=20):
    # Fit a regression model with a given l1_penalty on TRAIN data
    model = linear_model.Lasso(alpha=l1, normalize=True)
    model.fit(training[all_features], training['price'])
    count_nonzeros = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    if count_nonzeros > max_nonzeros:
        more_nonzeros.append(l1)
    if count_nonzeros < max_nonzeros:
        fewer_nonzeros.append(l1)
        

In [57]:
l1_penalty_min = max(more_nonzeros)
l1_penalty_max = min(fewer_nonzeros)

Exploring narrower range of l1_penalty between ‘l1_penalty_min’ and ‘l1_penalty_max’:

We look for the L1 penalty in this range that produces exactly the right number of nonzeros and also minimizes RSS on the VALIDATION set.

For l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
- Fit a regression model with a given l1_penalty on TRAIN data. As before, use "alpha=l1_penalty" and "normalize=True".
Measure the RSS of the learned model on the VALIDATION set
- Find the model that the lowest RSS on the VALIDATION set and has sparsity equal to ‘max_nonzeros’. (Again, take account of the intercept when counting the number of nonzeros.)



In [62]:
result = []
for l1 in np.linspace(l1_penalty_min,l1_penalty_max,20):
    # Fit a regression model with a given l1_penalty on TRAIN data
    model = linear_model.Lasso(alpha=l1, normalize=True)
    model.fit(training[all_features], training['price'])
    # Measure the RSS of the learned model on the VALIDATION set
    rss = sum((validation['price'] - model.predict(validation[all_features]))**2)
    # Find the model that the lowest RSS on the VALIDATION set and has sparsity equal to ‘max_nonzeros’.
    count_nonzeros = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    
    result.append({'l1': l1, 'rss': rss, 'count_nonzeros': count_nonzeros})


result = pd.DataFrame(result)

In [66]:
# l1_penalty value in our narrow range which has the lowest RSS on the VALIDATION set and has sparsity equal to ‘max_nonzeros’
selected_l1 = result[result['count_nonzeros']==max_nonzeros].sort_values('rss').reset_index(drop=True).iloc[0]['l1']

In [67]:
model_with_selected_l1 = linear_model.Lasso(alpha=selected_l1, normalize=True)
model_with_selected_l1.fit(training[all_features], training['price'])
[all_features[c] for c in [i for i, x in enumerate(model_with_selected_l1.coef_ !=0) if x]]

['bathrooms', 'sqft_living', 'waterfront', 'view', 'grade', 'yr_built']

### Implement LASSO solver via coordinate descent ###

- Write a function to normalize features
- Implement coordinate descent for LASSO
- Explore effects of L1 penalty

In [73]:
def get_numpy_data(df, features, output):
    df['constant'] = 1 
    features = ['constant'] + features
    features_df = df[features]
    
    feature_matrix = features_df.to_numpy()
    output_array = df['price'].to_numpy()

    return feature_matrix, output_array

def predict_output(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return predictions

In the house dataset, features vary wildly in their relative magnitude: ‘sqft_living’ is very large overall compared to ‘bedrooms’, for instance. As a result, weight for ‘sqft_living’ would be much smaller than weight for ‘bedrooms’. This is problematic because “small” weights are dropped first as l1_penalty goes up.

To give equal considerations for all features, we need to normalize features as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

In [74]:
def normalize_features(features):
    """
    normalize columns of a given feature matrix.
    :param features: columns of a feature matrix
    :return normalized features: normalized columns of given feature matrix
    :return norms: norms of orignal features
    """
    
    norms = np.linalg.norm(features, axis=0)
    normalized_features = features / norms
    
    return normalized_features, norms

#### Coordinate Descent: ####

At each iteration, we will fix all weights but weight i and find the value of weight i that minimizes the objective. 

All weights other than w[i] are held to be constant. We will optimize one w[i] at a time, circling through the weights multiple times.
- Pick a coordinate i
- Compute w[i] that minimizes the LASSO cost function
- Repeat the two steps for all coordinates, multiple times

#### Effect of L1 penalty ####

Consider a simple model with 2 features: ‘sqft_living’ and ‘bedrooms’. The output is ‘price’.

1. First, run get_numpy_data() (or equivalent) to obtain a feature matrix with 3 columns (constant column added). Use the entire ‘sales’ dataset for now.
2. Normalize columns of the feature matrix. Save the norms of original features as ‘norms’.
3. Set initial weights to [1,4,1].
4. Make predictions with feature matrix and initial weights.
5. Compute values of ro[i], where ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]


In [86]:
feature_matrix, output_array = get_numpy_data(sales, ['sqft_living', 'bedrooms'], 'price')
normalized_features, norms = normalize_features(feature_matrix)
initial_weights = [1,4,1]
predictions = predict_output(normalized_features, initial_weights)

In [94]:
ro_1 = sum(normalized_features[:,1] * (sales['price'] - predictions + initial_weights[1] * normalized_features[:,1]))
ro_2 = sum(normalized_features[:,2] * (sales['price'] - predictions + initial_weights[2] * normalized_features[:,2]))

Range of l1_penalty values that would not set w[1] zero, but would set w[2] zero
- -l1_penalty/2 <= ro_2 <= l1_penalty/2
- ro_1 > l1_penalty / 2

Equivalent to: 2*ro_2 <= l1_penalty < 2*ro_1

In [98]:
# Range of l1_penalty values that would not set w[1] zero, but would set w[2] zero
print(ro_2*2, " < l1_penalty < ", ro_1*2)

161933397.3324781  < l1_penalty <  175878941.64650303


Range of l1_penalty values that would set both w[1] and w[2] to zero
- -l1_penalty/2 <= ro_1 <= l1_penalty/2
- -l1_penalty/2 <= ro_2 <= l1_penalty/2

Equivalent to: l1_penalty >= 2*ro_1 (because ro_1 > ro_2)

In [99]:
# Range of l1_penalty values that would set both w[1] and w[2] zero
print("l1_penalty > ", ro_1*2)

l1_penalty >  175878941.64650303


=> So we can say that ro[i] quantifies the significance of the i-th feature: the larger ro[i] is, the more likely it is for the i-th feature to be retained.


#### Single Coordinate Descent Step ####
Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.


In [114]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    predictions = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = sum(feature_matrix[:,i] * (output - predictions + weights[i] * feature_matrix[:,i]))
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2.
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

In [115]:
# TEST: should print 0.425558846691
print(lasso_coordinate_descent_step(1, np.array([[3./sqrt(13),1./sqrt(10)],
                   [2./sqrt(13),3./sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.4255588466910251


#### Cyclical coordinate descent ####

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
- As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
- After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to the previous step.
Return weights

The function should accept the following parameters:
- Feature matrix
- Output array
- Initial weights
- L1 penalty
- Tolerance


In [116]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    # initialize variables
    weights = np.array(initial_weights)
    max_step = tolerance
    
    while max_step >= tolerance:
        step_sizes = []
        for i in range(len(weights)):
            new_weight = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            step_size = new_weight - weights[i]
            weights[i] = new_weight
            step_sizes.append(step_size)
        
        max_step = max(step_sizes)
            
    return weights

Let us now go back to the simple model with 2 features: ‘sqft_living’ and ‘bedrooms’. Using ‘get_numpy_data’ (or equivalent), extract the feature matrix and the output array from from the house dataframe. Then normalize the feature matrix using ‘normalized_features()’ function.

Using the following parameters, learn the weights on the sales dataset:
- Initial weights = all zeros
- L1 penalty = 1e7
- Tolerance = 1.0

In [118]:
feature_matrix, output_array = get_numpy_data(sales, ['sqft_living', 'bedrooms'], 'price')
normalized_features, norms = normalize_features(feature_matrix)
initial_weights = [0, 0, 0]
l1_penalty = 1e7
tolerance = 1

simple_model_weights = lasso_cyclical_coordinate_descent(normalized_features, output_array, initial_weights, l1_penalty, tolerance)

In [121]:
# RSS of the learned model on the normalized dataset
"{:e}".format(sum((output_array - predict_output(normalized_features, simple_model_weights))**2))

'1.630492e+15'

In [122]:
# Which features had weight zero at convergence?
simple_model_weights

array([21624995, 63157249,        0])

#### Evaluating LASSO fit with more features ####

Create a normalized feature matrix from the TRAINING data with the following set of features: bedrooms, bathrooms, sqft_living, sqft_lot, floors, waterfront, view, condition, grade, sqft_above, sqft_basement, yr_built, yr_renovated

First, learn the weights with l1_penalty=1e7, on the training data. Initialize weights to all zeros, and set the tolerance=1. Call resulting weights’ weights1e7’, you will need them later.

In [126]:
more_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']

feature_matrix, output_array = get_numpy_data(training, more_features, 'price')
normalized_features, norms = normalize_features(feature_matrix)
l1_penalty = 1e7
initial_weights = np.zeros(len(more_features)+1)
tolerance = 1
weights1e7 = lasso_cyclical_coordinate_descent(normalized_features, output_array, initial_weights, l1_penalty, tolerance)

In [140]:
# Which features had weight zero at convergence?
[(['constant'] + more_features)[w] for w in [i for i, x in enumerate(weights1e7 !=0) if x]]

['constant', 'sqft_living', 'waterfront', 'view']

In [143]:
l1_penalty=1e8
weights1e8 = lasso_cyclical_coordinate_descent(normalized_features, output_array, initial_weights, l1_penalty, tolerance)
[(['constant'] + more_features)[w] for w in [i for i, x in enumerate(weights1e8 !=0) if x]]


['constant']

In [144]:
l1_penalty = 1e4
tolerance = 5e5
weights1e4  = lasso_cyclical_coordinate_descent(normalized_features, output_array, initial_weights, l1_penalty, tolerance)
[(['constant'] + more_features)[w] for w in [i for i, x in enumerate(weights1e4 !=0) if x]]


['constant',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated']

#### Rescaling learned weights ####

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way. Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with original features:
- Store the norms of the original features to a vector called ‘norms’: features, norms = normalize_features(features)
- Run Lasso on the normalized features and obtain a ‘weights’ vector
- Compute the weights for the original features by performing element-wise division, i.e. weights_normalized = weights / norms

Now, we can apply weights_normalized to the test data, without normalizing it!


In [151]:
normalized_weights1e4 = weights1e4 / norms
normalized_weights1e7 = weights1e7 / norms
normalized_weights1e8 = weights1e8 / norms

In [155]:
test_feature_matrix, test_output_array = get_numpy_data(testing, more_features, 'price')


In [156]:
rss1e4 = sum((test_output_array - predict_output(test_feature_matrix, normalized_weights1e4))**2)
rss1e7 = sum((test_output_array - predict_output(test_feature_matrix, normalized_weights1e7))**2)
rss1e8 = sum((test_output_array - predict_output(test_feature_matrix, normalized_weights1e8))**2)


In [158]:
for rss in [rss1e4, rss1e7, rss1e8]:
    print("{:e}".format(rss))

1.290853e+14
1.631036e+14
2.847189e+14
